# Vector Retriever 성능 분석 (Hit@k & MRR)

이 노트북은 **VectorRetriever**의 의미론적 검색 성능을 정량적으로 평가합니다.

In [ ]:
# ✅ 실험 파라미터 설정
SAMPLE_SIZE = 50
K_VALUES = [1, 3, 5]

# 환경 설정
import sys
import os
from dotenv import load_dotenv
import pandas as pd
import json
import random

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

load_dotenv(os.path.join(project_root, ".env"))
print(f"Project Root: {project_root}")

In [ ]:
from src.retrieval.vector_retriever import VectorRetriever

retriever = VectorRetriever(version="v3", collection_name="care_guides")
print("✅ VectorRetriever initialized.")

## 1. 데이터셋 로드

In [ ]:
dataset_path = os.path.join(project_root, "data/v3/golden_dataset.json")

if os.path.exists(dataset_path):
    with open(dataset_path, "r", encoding="utf-8") as f:
        FULL_DATASET = json.load(f)
    print(f"📚 Loaded {len(FULL_DATASET)} test cases.")
else:
    print("⚠️ Dataset not found. Using mock.")
    FULL_DATASET = [{"query": "메인쿤 특징", "expected_keyword": "메인 쿤"}]

test_dataset = random.sample(FULL_DATASET, min(SAMPLE_SIZE, len(FULL_DATASET)))
print(f"🧪 Selected {len(test_dataset)} samples.")

## 2. 평가 로직 구현 (Hit@k, MRR)

In [ ]:
async def evaluate_vector(dataset, k_values):
    results_summary = {k: 0 for k in k_values}
    total_mrr = 0
    log_data = []
    
    max_k = max(k_values)
    
    for item in dataset:
        query = item.get("query")
        target = item.get("expected_keyword", item.get("target"))
        
        if not target: continue
        
        specialist = item.get("specialist")
        search_results = await retriever.search(query, specialist=specialist, limit=max_k)
        
        found_rank = None
        for rank, res in enumerate(search_results):
            content = (
                res.get('name_ko', '') + " " + 
                res.get('title_refined', '') + " " + 
                res.get('text', '')
            ).lower()
            
            if target.lower() in content:
                found_rank = rank + 1
                break
        
        reciprocal_rank = 0
        hit_status = "❌"
        
        if found_rank:
            reciprocal_rank = 1 / found_rank
            hit_status = f"✅ (Rank {found_rank})"
            
            for k in k_values:
                if found_rank <= k:
                    results_summary[k] += 1
        
        total_mrr += reciprocal_rank
        
        log_data.append({
            "Query": query,
            "Target": target,
            "Result": hit_status,
            "MRR": round(reciprocal_rank, 4)
        })
        
    # 최종 집계
    total_count = len(dataset)
    metrics = {
        f"Hit@{k}": round(count / total_count, 4) 
        for k, count in results_summary.items()
    }
    metrics["MRR"] = round(total_mrr / total_count, 4)
    
    return metrics, pd.DataFrame(log_data)

print("Evaluator ready.")

In [ ]:
metrics, df_log = await evaluate_vector(test_dataset, K_VALUES)

print("="*40)
print("   📊 Vector Evaluation Results")
print("="*40)
for k, v in metrics.items():
    print(f"{k:<10}: {v}")
print("="*40)

display(df_log.head(10))